In [ ]:
var scenario = "IRU";

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [ ]:
Workspace.Initialize(x => x.FromSource(DataSource));

# Import Scenario Data

In [ ]:
await UploadDimensionsAsync<YieldCurve>("../Files/800.Parameters/YieldCurveScenario.csv")

In [ ]:
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv")

In [ ]:
//Define partition
var args = new ImportArgs("CH", 2020, 12, Periodicity.Quarterly, scenario, ImportFormats.Actual);
var partition = DataSource.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year && 
                                                                                    x.Month == args.Month && x.Scenario == args.Scenario);
if(partition == null) ApplicationMessage.Log(Error.PartitionNotFound);

var previousPeriodPartition = Workspace.Query<PartitionByReportingNodeAndPeriod>().FirstOrDefault(x => x.ReportingNode == args.ReportingNode && x.Year == args.Year - 1 && 
                                                                                    x.Month == MonthInAYear && x.Scenario == args.Scenario);
if(previousPeriodPartition == null) ApplicationMessage.Log(Error.PartitionNotFound);
DataSource.Query<PartitionByReportingNodeAndPeriod>()

In [ ]:
partition